In [1]:
import torch
import random
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import torch.nn as nn
import torch.nn.functional as F

import math
import time

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


# Load the data

In [3]:
root_path = "/scratch/04703/sravula/UTAFSDataNew/new_data"
chip_num = 1

In [4]:
import utils

data_dict = utils.grab_chip_data(root_path, chip_num, resample=False)

In [5]:
data_dict.keys()

dict_keys(['gt_matrix', 'gt_freqs', 'vf_matrix', 'y_matrix', 'y_freqs'])

## Check the data matrices converted from touchstone to np

In [6]:
print(data_dict['gt_matrix'].shape)
print(data_dict['vf_matrix'].shape)
print(data_dict['y_matrix'].shape)

(501, 4, 4, 2)
(501, 4, 4, 2)
(18, 4, 4, 2)


In [7]:
print(data_dict['gt_matrix'].dtype)
print(data_dict['vf_matrix'].dtype)
print(data_dict['y_matrix'].dtype)

float32
float32
float32


In [8]:
print(data_dict['gt_freqs'].shape)
print(data_dict['y_freqs'].shape)

(501,)
(18,)


In [9]:
print(data_dict['gt_freqs'].dtype)
print(data_dict['y_freqs'].dtype)

float64
float64


## Convert the raw data matrices to only the unique S-parameters

In [10]:
vf_data = utils.matrix_to_sparams(data_dict['vf_matrix'])
gt_data = utils.matrix_to_sparams(data_dict['gt_matrix'])
y_data = utils.matrix_to_sparams(data_dict['y_matrix'])

In [11]:
print(vf_data.shape)
print(gt_data.shape)
print(y_data.shape)

(10, 2, 501)
(10, 2, 501)
(10, 2, 18)


In [12]:
print(vf_data.dtype)
print(gt_data.dtype)
print(y_data.dtype)

float32
float32
float32


In [13]:
gt_freqs = data_dict['gt_freqs']
y_freqs = data_dict['y_freqs']

In [14]:
print(gt_freqs.shape)
print(y_freqs.shape)

(501,)
(18,)


In [15]:
print(gt_freqs.dtype)
print(y_freqs.dtype)

float64
float64
